# Membuat model jaringan (graph)
pada tahapan ini dilakukan pembuatan jaringan dengan menggunakan attribut `username` dan `mentioned`.
setiap baris data akan memiliki `username` namun pada `mentioned` bisa terdapat lebih dari 1 username.
example:
```json
{
    "username": "kurniawan",
    "mentioned": ["niwa", "ronaldi", "purnama"]
}
```
dari contoh data tersebut maka terdapat 3 `edges` yang dapat dihasilkan
```json
{
    "edges": [{
        "source": "kurniawan",
        "target": "niwa"
    },{
        "source": "kurniawan",
        "target": "ronaldi"
    },{
        "source": "kurniawan",
        "target": "purnama
    }]
}
```

In [8]:
# import library yang digunakan
import pandas as pd
import networkx as nx
import json

# untuk memperbaiki array yang dianggap sebagai str
from ast import literal_eval

#### Membaca data dari proses sebelumnya
pada tahapan ini hanya mengambil attribut `username` dan `mentioned`

In [3]:
data = pd.read_csv("../out/01-preprocessing.tsv", sep="\t")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5428 entries, 0 to 5427
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   raw_content         5428 non-null   object
 1   username            5428 non-null   object
 2   tweet_published_at  5428 non-null   object
 3   type                5428 non-null   object
 4   cleaned_content     5428 non-null   object
 5   mentioned           5428 non-null   object
dtypes: object(6)
memory usage: 254.6+ KB


## Membuat Graph

In [4]:
G = nx.DiGraph()

In [5]:
for x in data[['username', 'mentioned']].iloc:
    source = x.username
    for target in literal_eval(x.mentioned):
        if G.has_edge(source, target):
            G[source][target]['weight'] += 1
        else:
            G.add_edge(source, target, weight=1)

mengexport graph ke dalam bentuk `.gexf` dan `.json`

In [7]:
nx.write_gexf(G, "../out/02-network.gexf")

In [9]:
with open("../out/02-network.json", "w") as f:
    json.dump(nx.node_link_data(G), f)

# Membuat model Latent Dirichlet Allocation